In [1]:
import torch
import gc
import os
from fft_conv_pytorch import fft_conv, FFTConv2d

# Deconvolution Related Functions

In [2]:
def fft_convolve_images(image, kernel_stack, summation = False):
    # Using this will result in worse performance. Only use this when you are out of memory.
    
    # image is the image to be convolved
    # kernel_stack is the stack of kernels to be convolved with the image
    
    # The image and the kernel stack should be the same size
    # The kernel stack should be in the format [depth, x, y]
    return_tensor = torch.fft.ifftshift(torch.fft.ifft2(torch.mul(torch.fft.fft2(image), torch.fft.fft2(torch.div(kernel_stack, torch.sum(kernel_stack, dim = (1,2)).unsqueeze(1).unsqueeze(2)))), dim = (1,2))) # Convolve the image with the kernel stack
        
    if summation == True:
        return_tensor = torch.sum(return_tensor, 0) # Sum all of the convolutions
    
    return return_tensor

In [3]:
def richardson_lucy(original_image, f, b, iterations, device):
    # e_i is the image to be deconvolved
    # oringinal_image is the image that was convolved with the kernel
    # f is the forward projector
    # b is the backward projector
    with torch.no_grad():
        kernel_size_f = f.shape[0]
        func1 = torch.nn.Conv2d(1, 1, kernel_size=kernel_size_f, stride=1, padding = 'same', device = device)
        func1.weight = f
        
        kernel_size_b = b.shape[0]
        func2 = torch.nn.Conv2d(1, 1, kernel_size=kernel_size_b, stride=1, padding = 'same', device = device)
        func2.weight = b
        
        e_i = func2(original_image.detach())
        
        for i in range(int(iterations)):
            denom = func1(e_i)
            fraction = torch.div(original_image.detach(), denom)
            
            del denom
            gc.collect()
            if device == 'cuda':
                torch.cuda.empty_cache()
                
            factor = func2(fraction)
            
            del fraction
            gc.collect()
            if device == 'cuda':
                torch.cuda.empty_cache()
            
            e_i = torch.mul(e_i, factor)
            
            del factor
            gc.collect()
            if device == 'cuda':
                torch.cuda.empty_cache()
    
    return e_i

In [4]:
def rl_deconv(frames, PSF, PSF_bp, device, iterations = 10, path = r"./", cpu_option = 'cpu', temp_storage = False):
    # Take the whole lenslet image & the whole PSFs and apply the Richardson-Lucy algorithm to it
    # Frames should be in the format of [time, x, y]
    # PSF should be in the format of [depths, x, y]
    # Output should be in the format of [time, depth, x, y]
    # cpu_option is the device that does the division and the multiplication.
    # temp_storage dictates how the temporary storage is handled. If it is set to True, then the temporary storage is saved to the disk. If it is set to False, then the temporary storage is saved to the RAM.
    
    # So, FFTConv2D CAN work, it's just that we have to process one channel at a time and empty out cache after each iteration.
    # The problem is that the memory is not being freed up after each iteration. This is a problem with the way that the memory is being handled.
    
    with torch.autocast(device_type=device):
        with torch.no_grad():
            conv2d = FFTConv2d(1, 1, kernel_size = (len(PSF_bp[0,:,0]), len(PSF_bp[0,0,:])), stride = 1, padding = 'same', bias = False).to(device)
            
            print("Initializing the convolutional layer for the Richardson-Lucy algorithm")
            
            if type(frames) != torch.Tensor:
                frames = torch.tensor(frames)
                gc.collect()
            
            if frames.dtype != torch.float16:
                frames = frames.to(torch.float16)
                gc.collect()
                
            if device == 'cuda':
                torch.cuda.empty_cache()
            
            for k in range(len(frames[:,0,0])):
                # Each time step
                #print("Creating projected volume and image for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])))
                
                original_image = frames[k,:,:].clone().detach().to(cpu_option) # Format: (x,y)
                projected_volume = torch.empty([len(PSF[:,0,0]), len(PSF[0,:,0]), len(PSF[0,0,:])], dtype = torch.float16).detach().to(cpu_option) # Format: (depth, x,y)
                projected_image = original_image.clone().detach() # Format: (x,y), we do not want the original image to be modified
                
                print("Finished initialization for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])) + ", now creating the projected volume and image for the frame")
                
                gc.collect()
                if device == 'cuda':
                    torch.cuda.empty_cache()
                
                for i in range(len(PSF[:,0,0])): # Save the projected volume to the CPU memory, but use gpu to do the calculations (if possible)
                    #print("Now creating projected volume " + str(i+1) + " out of " + str(len(PSF[:,0,0])) + " for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])))
                    conv2d.weights = torch.nn.parameter.Parameter(torch.div(PSF_bp[i,:,:].detach(), torch.sum(PSF_bp[i,:,:].detach())).detach().to(device).unsqueeze(0).unsqueeze(0))
                    temp_vol = conv2d(original_image.detach().unsqueeze(0).unsqueeze(0).to(device)).squeeze(0).squeeze(0)
                    
                    if temp_storage == True:
                        torch.save(temp_vol, path +'tempvol'+ '_' + str(i) + '.pt')
                        
                    if temp_storage == False:
                        projected_volume[i,:,:] = temp_vol.to(cpu_option)
                        
                    del temp_vol
                    gc.collect()

                    if device == 'cuda':
                        torch.cuda.empty_cache()
                
                print("Finished creating the projected volume and image for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])) + ", now processing the iterations")
                
                for j in range(iterations):
                    print("Now processing iteration " + str(j+1) + " out of " + str(iterations) + " for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])))
                    # Schema:
                    # Calculate the projected image (Sum of o_i * PSF)
                    # Divide the original image by the projected image (original_image / projected_image), (should be done on the cpu?)                
                    # Convolve the ratio with the backward projector (ratio * PSF_bp)
                    # Multiply the original image by the result of the convolution (original_image * result)
                    # Repeat until the iterations are done
                    
                    # Denominator results in a projected image
                    
                    for i in range(len(PSF[:,0,0])):
                        conv2d.weight = torch.nn.parameter.Parameter(torch.div(PSF[i,:,:].detach(), PSF[i,:,:].detach()).detach().to(device).unsqueeze(0).unsqueeze(0))
                        
                        if temp_storage == True:
                            temp_vol = torch.load(path + 'tempvol' + '_' + str(i) + '.pt').to(device)
                            projected_image += conv2d(temp_vol.unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0).to(cpu_option)
                            del temp_vol
                            gc.collect()
                        
                        projected_image += conv2d(projected_volume[i,:,:].unsqueeze(0).unsqueeze(0).to(device)).squeeze(0).squeeze(0).to(cpu_option)
                        if temp_storage == False:
                            projected_image += conv2d(projected_volume[i,:,:].unsqueeze(0).unsqueeze(0).to(device)).squeeze(0).squeeze(0).to(cpu_option)
                        
                        if device == 'cuda':
                            torch.cuda.empty_cache()
                    
                    # We have the projected image, now take the ratio. The projected image is in 'device'.
                    
                    image_ratio = torch.nan_to_num(torch.div(original_image,projected_image).to(cpu_option), 1) # If the projected image is 0, then the ratio is 1
                    
                    # Convolve the ratio with the backward projector
                    
                    factor = torch.empty([len(PSF[:,0,0]),len(PSF[0,:,0]), len(PSF[0,0,:])], dtype = torch.float16).to(cpu_option) # Format: (depth,x,y)\
                    
                    for i in range(len(PSF_bp[:,0,0])):
                        #print("Now creating multiplying factor " + str(i+1) + " out of " + str(len(PSF_bp[:,0,0])) + " for iteration " + str(j+1) + " out of " + str(iterations) + " for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])))
                        conv2d.weight = torch.nn.parameter.Parameter(torch.div(PSF_bp[i,:,:].detach(), torch.sum(PSF_bp[i,:,:].detach())).detach().to(device).unsqueeze(0).unsqueeze(0).to(device))
                        factor_results = conv2d(image_ratio.to(device).unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0).to(cpu_option)
                        
                        if temp_storage == True:
                            torch.save(factor_results, path + 'factor' + '_' + str(i) + '.pt')
                            
                        elif temp_storage == False:
                            factor[i,:,:] = factor_results.to(cpu_option)
                        
                        del factor_results
                        gc.collect()
                        
                    del image_ratio # We don't need this anymore
                    gc.collect()
                    
                    if device == 'cuda':
                        torch.cuda.empty_cache()
                    
                    # We have the new factor, now multiply it by the original volume
                    if temp_storage == True:
                        for i in range(len(PSF[:,0,0])):
                            temp_vol = torch.load(path + 'tempvol' + '_' + str(i) + '.pt').to(cpu_option)
                            temp_factor = torch.load(path + 'factor' + '_' + str(i) + '.pt').to(cpu_option)
                            torch.save(torch.mul(temp_vol, temp_factor), path +'tempvol'+ '_' + str(i) + '.pt')
                            
                            del temp_vol, temp_factor
                            gc.collect()
                            
                            if device == 'cuda':
                                torch.cuda.empty_cache()
                    
                    if temp_storage == False:
                        projected_volume[:,:,:] = torch.mul(projected_volume[:,:,:].to(cpu_option), factor[:,:,:].to(cpu_option)).to(cpu_option)
                
                print("Finished processing frame " + str(k+1) + " out of " + str(len(frames[:,0,0])) + ", now saving the results")
                
                if temp_storage == False:
                    torch.save(projected_volume, path + str(k) + '.pt')
                    
                for i in range(len(PSF[:,0,0])):
                    if temp_storage == True:
                        os.rename(path + 'tempvol' + '_' + str(i) + '.pt', path + str(k) + '_' + str(i) + '.pt')
            
            del projected_volume, projected_image, original_image, factor
            gc.collect()
            
            if device == 'cuda':
                torch.cuda.empty_cache()
            
    return None

In [5]:
def rl2_deconv(frames, PSF, PSF_bp, device, iterations = 10, path = r"./", cpu_option = 'cpu', temp_storage = False):
    # Take the whole lenslet image & the whole PSFs and apply the Richardson-Lucy algorithm to it
    # Frames should be in the format of [time, x, y]
    # PSF should be in the format of [depths, x, y]
    # Output should be in the format of [time, depth, x, y]
    # cpu_option is the device that does the division and the multiplication.
    # temp_storage dictates how the temporary storage is handled. If it is set to True, then the temporary storage is saved to the disk. If it is set to False, then the temporary storage is saved to the RAM.
    
    # So, FFTConv2D CAN work, it's just that we have to process one channel at a time and empty out cache after each iteration.
    # The problem is that the memory is not being freed up after each iteration. This is a problem with the way that the memory is being handled.
    
    with torch.autocast(device_type=device):
        with torch.no_grad():
            conv2d = FFTConv2d(1, 1, kernel_size = (len(PSF_bp[0,:,0]), len(PSF_bp[0,0,:])), stride = 1, padding = 'same', bias = False).to(device)
            
            print("Initializing the convolutional layer for the Richardson-Lucy algorithm")
            
            if type(frames) != torch.Tensor:
                frames = torch.tensor(frames)
                gc.collect()
            
            if frames.dtype != torch.float16:
                frames = frames.to(torch.float16)
                gc.collect()
                
            if device == 'cuda':
                torch.cuda.empty_cache()
            
            for k in range(len(frames[:,0,0])):
                # Each time step
                #print("Creating projected volume and image for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])))
                
                original_image = frames[k,:,:].clone().detach().to(cpu_option) # Format: (x,y)
                projected_volume = torch.empty([len(PSF[:,0,0]), len(PSF[0,:,0]), len(PSF[0,0,:])], dtype = torch.float16).detach().to(cpu_option) # Format: (depth, x,y)
                projected_image = original_image.clone().detach() # Format: (x,y), we do not want the original image to be modified
                
                print("Finished initialization for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])) + ", now creating the projected volume and image for the frame")
                
                gc.collect()
                if device == 'cuda':
                    torch.cuda.empty_cache()
                
                for i in range(len(PSF[:,0,0])): # Save the projected volume to the CPU memory, but use gpu to do the calculations (if possible)
                    #print("Now creating projected volume " + str(i+1) + " out of " + str(len(PSF[:,0,0])) + " for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])))
                    conv2d.weights = torch.nn.parameter.Parameter(torch.div(PSF_bp[i,:,:].detach(), torch.sum(PSF_bp[i,:,:].detach())).detach().to(device).unsqueeze(0).unsqueeze(0))
                    temp_vol = conv2d(original_image.detach().unsqueeze(0).unsqueeze(0).to(device)).squeeze(0).squeeze(0)
                    
                    if temp_storage == True:
                        torch.save(temp_vol, path +'tempvol'+ '_' + str(i) + '.pt')
                        
                    if temp_storage == False:
                        projected_volume[i,:,:] = temp_vol.to(cpu_option)
                        
                    del temp_vol
                    gc.collect()

                    if device == 'cuda':
                        torch.cuda.empty_cache()
                
                print("Finished creating the projected volume and image for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])) + ", now processing the iterations")
                
                for j in range(iterations):
                    print("Now processing iteration " + str(j+1) + " out of " + str(iterations) + " for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])))
                    # Schema:
                    # Calculate the projected image (Sum of o_i * PSF)
                    # Divide the original image by the projected image (original_image / projected_image), (should be done on the cpu?)                
                    # Convolve the ratio with the backward projector (ratio * PSF_bp)
                    # Multiply the original image by the result of the convolution (original_image * result)
                    # Repeat until the iterations are done
                    
                    # Denominator results in a projected image
                    
                    for i in range(len(PSF[:,0,0])):
                        conv2d.weight = torch.nn.parameter.Parameter(torch.div(PSF[i,:,:].detach(), PSF[i,:,:].detach()).detach().to(device).unsqueeze(0).unsqueeze(0))
                        
                        if temp_storage == True:
                            temp_vol = torch.load(path + 'tempvol' + '_' + str(i) + '.pt').to(device)
                            projected_image += conv2d(temp_vol.unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0).to(cpu_option)
                            del temp_vol
                            gc.collect()
                        
                        projected_image += conv2d(projected_volume[i,:,:].unsqueeze(0).unsqueeze(0).to(device)).squeeze(0).squeeze(0).to(cpu_option)
                        if temp_storage == False:
                            projected_image += conv2d(projected_volume[i,:,:].unsqueeze(0).unsqueeze(0).to(device)).squeeze(0).squeeze(0).to(cpu_option)
                        
                        if device == 'cuda':
                            torch.cuda.empty_cache()
                    
                    # We have the projected image, now take the ratio. The projected image is in 'device'.
                    
                    image_ratio = torch.nan_to_num(torch.div(original_image,projected_image).to(cpu_option), 1) # If the projected image is 0, then the ratio is 1
                    
                    # Convolve the ratio with the backward projector
                    
                    factor = torch.empty([len(PSF[:,0,0]),len(PSF[0,:,0]), len(PSF[0,0,:])], dtype = torch.float16).to(cpu_option) # Format: (depth,x,y)\
                    
                    for i in range(len(PSF_bp[:,0,0])):
                        #print("Now creating multiplying factor " + str(i+1) + " out of " + str(len(PSF_bp[:,0,0])) + " for iteration " + str(j+1) + " out of " + str(iterations) + " for frame " + str(k+1) + " out of " + str(len(frames[:,0,0])))
                        conv2d.weight = torch.nn.parameter.Parameter(torch.div(PSF_bp[i,:,:].detach(), torch.sum(PSF_bp[i,:,:].detach())).detach().to(device).unsqueeze(0).unsqueeze(0).to(device))
                        factor_results = conv2d(image_ratio.to(device).unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0).to(cpu_option)
                        
                        if temp_storage == True:
                            torch.save(factor_results, path + 'factor' + '_' + str(i) + '.pt')
                            
                        elif temp_storage == False:
                            factor[i,:,:] = factor_results.to(cpu_option)
                        
                        del factor_results
                        gc.collect()
                        
                    del image_ratio # We don't need this anymore
                    gc.collect()
                    
                    if device == 'cuda':
                        torch.cuda.empty_cache()
                    
                    # We have the new factor, now multiply it by the original volume
                    if temp_storage == True:
                        for i in range(len(PSF[:,0,0])):
                            temp_vol = torch.load(path + 'tempvol' + '_' + str(i) + '.pt').to(cpu_option)
                            temp_factor = torch.load(path + 'factor' + '_' + str(i) + '.pt').to(cpu_option)
                            torch.save(torch.mul(temp_vol, temp_factor), path +'tempvol'+ '_' + str(i) + '.pt')
                            
                            del temp_vol, temp_factor
                            gc.collect()
                            
                            if device == 'cuda':
                                torch.cuda.empty_cache()
                    
                    if temp_storage == False:
                        projected_volume[:,:,:] = torch.mul(projected_volume[:,:,:].to(cpu_option), factor[:,:,:].to(cpu_option)).to(cpu_option)
                
                print("Finished processing frame " + str(k+1) + " out of " + str(len(frames[:,0,0])) + ", now saving the results")
                
                if temp_storage == False:
                    torch.save(projected_volume, path + str(k) + '.pt')
                    
                for i in range(len(PSF[:,0,0])):
                    if temp_storage == True:
                        os.rename(path + 'tempvol' + '_' + str(i) + '.pt', path + str(k) + '_' + str(i) + '.pt')
            
            del projected_volume, projected_image, original_image, factor
            gc.collect()
            
            if device == 'cuda':
                torch.cuda.empty_cache()
            
    return None

SyntaxError: '(' was never closed (995219146.py, line 53)

# Go get 'em

In [6]:
results = rl_deconv(torch.load(r'./test_images/Frames/picture.pt'), torch.load(r'./test_models/forward.pt'), torch.load(r'./test_models/backward.pt'), 'cuda', 10, r'./test_images/', 'cpu', False)

Initializing the convolutional layer for the Richardson-Lucy algorithm
Finished initialization for frame 1 out of 1, now creating the projected volume and image for the frame
Finished creating the projected volume and image for frame 1 out of 1, now processing the iterations
Now processing iteration 1 out of 10 for frame 1 out of 1
torch.Size([20, 8192, 8192]) torch.Size([20, 8192, 8192])
Now processing iteration 2 out of 10 for frame 1 out of 1
